#### 데이터 가져오기

conda install shapely
conda install geopy

필요

In [1]:
# 이번 시간에는 저장된 학원 위경도 데이터를 가지고 학원가 클러스터링을 지도에 표시해보도록 합니다. 
import pandas as pd 
import numpy as np
from sklearn.cluster import DBSCAN

In [2]:
#한글 윈도우에서는 cp949 인코딩을 사용하기 때문에 열어보기 위해서 인코딩 설정을 cp949로 합니다.
df = pd.read_csv('분당구데이터.csv', encoding='cp949')

In [3]:
df.head()

,학원교습소명,학원명,개설일자,등록일자,등록상태명,정원합계,일시수용능력인원합계,분야명,교습계열명,교습과정명,도로명우편번호,도로명주소,도로명상세주소,구,lat,lon
0,학원,(2관)기찬에듀기찬수학학원,20200820,20200820,개원,240,95,입시.검정 및 보습,보통교과,보습,13555,경기도 성남시 분당구 정자일로 234,/ 305호 (정자동/ 태남프라자),분당구,37.371212,127.106282
1,학원,(2관)짱솔학원,20151124,20151124,개원,150,114,입시.검정 및 보습,보통교과,보습,13601,경기도 성남시 분당구 불정로 252,/ 401호/402호 (수내동),분당구,37.366216,127.126642
2,학원,(분당점)더원학원,20220610,20220610,개원,300,143,입시.검정 및 보습,보통교과,보습,13601,경기도 성남시 분당구 불정로 254,/ 4층 (수내동/ 삼평프라자),분당구,37.366395,127.126822
3,학원,(주)그래디언트학원,20111205,20111205,개원,75,72,입시.검정 및 보습,보통교과,보습,13558,경기도 성남시 분당구 성남대로331번길 3-13,/ 606호 (정자동/ 대명제스트빌딩),분당구,37.364393,127.107696
4,학원,(주)아이에스이영어원격학원,20120720,20120720,개원,27,100,입시.검정 및 보습,보통교과,보습,13591,경기도 성남시 분당구 황새울로360번길 21,/ 604호 일부 (서현동/ 분당서현신영팰리스타워),분당구,37.385710,127.125705


###  2. Folium 을 이용한 클러스터링 표시 

In [4]:
# folium 은 인터랙티브한 지도 시각화 라이브러리 입니다. 
# MarkerCluster를 이용합니다. 


import folium
from folium.plugins import MarkerCluster

#지도를 표시해주며 지도에 표시될 중심 좌표는 location 에 세팅해줍니다. 
m = folium.Map(
    location=[37.3800036, 127.1184932],
    zoom_start=15
)

# 지도 표시에 필요한 데이터 위경도와 학원명을 가지고 와 coords 변수에 저장합니다. 
coords = df[['lat' , 'lon', '학원명']]


#MarkerCluster 에 표시할 지도를 등록해줍니다.
marker_cluster = MarkerCluster().add_to(m)

for lat, long, name in zip(coords['lat'], coords['lon'], coords['학원명']):
    marker = folium.Marker([lat, long], popup =  name + ''  + str(lat) + ',' + str(long) , icon = folium.Icon(color="green"))
    marker.add_to(marker_cluster)
m


### 3. DBSCAN 을 이용한 클러스터링 표시

##### DBSCAN 알고리즘

- DBSCAN : 밀도 기반 공간 클러스터링 <br>
특정 요소가 클러스터에 속하는 경우 해당 클러스터 내 다른 많은 요소와 가까운 위치에 있어야 한다는 아이디어를 전제로 하며, <br>
이 계산을 위해 직경과(eps) 와 최소 요소를 설정합니다.

- DBSCAN 사용 이유 : https://osf.io/preprints/socarxiv/nzhdc/   논문 참고 <br>
 K-means 은 분산을 최소로 하지, 측지학적 거리를 최소화 하지 않음 <br>
 적도에서 상당히 떨어진 위도에서는 지구는 둥글기 때문에 데이터 왜곡이 일어날 수 있다. <br> 
 
- haversine : 두 위경도 좌표 사이의 거리를 구할 때 사용하는 공식 <br> 

- scikit-learn’s haversine metric 을 이용시에는 위경도 및 직경을 radian unit 으로 변경해야 함. 



##### DBSCAN 초기 파라미터

#### Parameter

<b> 100m 직경 내에 7개 이상일 경우 클러스터링 되도록 설정함 </b>
- 위경도 : np.radians 함수 이용 (degree to radians)
- eps : earth radius in kilometers = 6371.008   
        1 radius = 6371.008 km
        1 / 6371.008  radius = 1km 
        
        100 m 이므로 0.1 km
        따라서 0.1 / 6371.008 
        
-  min_samples = 7

In [5]:
df_location = df[['lat' , 'lon']]

In [6]:
# metric : 피쳐 배열에서 인스턴스 간 거리를 계산할 때 사용함 

dbscan = DBSCAN(eps=0.1/6371.008, min_samples=7, algorithm='ball_tree', metric='haversine').fit(np.radians(df_location))

In [7]:
print(dbscan.labels_)

[ 0  1  1 ... -1 11 27]


In [8]:
df_location['cluster'] = dbscan.labels_

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_27592\1450822654.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_location['cluster'] = dbscan.labels_


In [9]:
df_location.drop(df_location[df_location['cluster'] == -1].index, inplace=True)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_27592\1683935843.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_location.drop(df_location[df_location['cluster'] == -1].index, inplace=True)


In [10]:
cluster_labels = df_location['cluster']
num_clusters = len(set(cluster_labels))
num_clusters

52

In [11]:
print(set(cluster_labels))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51}


In [12]:
cluster_labels == 0

0        True
1       False
2       False
3       False
4       False
        ...  
2118    False
2119     True
2121    False
2123    False
2124    False
Name: cluster, Length: 1760, dtype: bool

In [13]:
df_location[cluster_labels == 0]

,lat,lon,cluster
0,37.371212,127.106282,0
9,37.370281,127.107517,0
13,37.371212,127.106282,0
37,37.371212,127.106282,0
49,37.371557,127.106288,0
...,...,...,...
2094,37.371557,127.106288,0
2095,37.371212,127.106282,0
2111,37.371212,127.106282,0
2112,37.371212,127.106282,0


In [14]:
df_location_numpy = df_location[[ 'lat' , 'lon']].to_numpy()
clusters = pd.Series([df_location_numpy[cluster_labels == n] for n in range(num_clusters)])

In [15]:
clusters

0     [[37.37121219, 127.106281974], [37.37028055, 1...
1     [[37.366216199, 127.126641894], [37.366395367,...
2     [[37.364393218, 127.107695832], [37.364393218,...
3     [[37.385710025, 127.125705411], [37.385710025,...
4     [[37.367294923, 127.107698321], [37.366891804,...
5     [[37.36876245, 127.112228977], [37.368791269, ...
6     [[37.374923464, 127.118566987], [37.373482279,...
7     [[37.39003334, 127.088960872], [37.390655228, ...
8     [[37.399498007, 127.12234419], [37.399493346, ...
9     [[37.377133025, 127.115477451], [37.377383949,...
10    [[37.383128014, 127.125512449], [37.383128014,...
11    [[37.336663452, 127.116234193], [37.337788994,...
12    [[37.403965518, 127.116139678], [37.404241633,...
13    [[37.391312688, 127.077357796], [37.391963622,...
14    [[37.349715117, 127.111735377], [37.349688512,...
15    [[37.379709372, 127.130164702], [37.380881743,...
16    [[37.405673773, 127.12670442], [37.405133113, ...
17    [[37.39102163, 127.126382155], [37.3915537

In [16]:
from shapely.geometry import MultiPoint
from geopy.distance import great_circle



def get_centermost_point(cluster):
    
    # 각 클러스터의 center x, y 좌표 return
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    
    # great_circle 는 좌표 간 거리를 계산해 주며 .m 의 의미는 좌표간 거리를 meter 단위로 변환해준다
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)

    
    return tuple(centermost_point)

In [17]:
# 각 클러스터의 중심점에서 가장 가까운 위경도 좌표를 찾는다. 
centermost_points = clusters.apply(get_centermost_point)

centermost_points

0      (37.37121219, 127.106281974)
1     (37.366634886, 127.127160327)
2     (37.364393218, 127.107695832)
3     (37.385710025, 127.125705411)
4     (37.367294923, 127.107698321)
5      (37.36876245, 127.112228977)
6     (37.374089311, 127.119468536)
7     (37.389839279, 127.089800678)
8      (37.399447483, 127.12282528)
9     (37.377133025, 127.115477451)
10    (37.383128014, 127.125512449)
11    (37.337387125, 127.115823862)
12    (37.403831024, 127.115910772)
13       (37.39136194, 127.0766354)
14    (37.350395806, 127.110801573)
15    (37.380350582, 127.129417363)
16     (37.40514391, 127.126698305)
17    (37.391553773, 127.125713085)
18     (37.36999036, 127.123626496)
19     (37.38367221, 127.121996097)
20    (37.394985744, 127.126130773)
21    (37.377366332, 127.133670401)
22    (37.385486867, 127.111442367)
23    (37.361322979, 127.104866991)
24    (37.368163665, 127.106311359)
25    (37.371958448, 127.139593414)
26    (37.374359756, 127.136415698)
27    (37.376744011, 127.112

In [18]:
# unzip the list of centermost points (lat, lon) tuples into separate lat and lon lists
# centermost_points 을 unzip 하여 각각의 개별값으로 분리한다. 
lats, lons = zip(*centermost_points)

rep_points = pd.DataFrame({'lats':lats, 'lon':lons})
rep_points.tail()

,lats,lon
47,37.407371,127.145510
48,37.399463,127.126908
49,37.405843,127.118311
50,37.385583,127.120945
51,37.410807,127.129859


In [19]:
#클러스터의 중심으로 지정된 위경도 값(rep_points)과 동일한 위경도 값을 가진 데이터를 df 에서 가지고 온다.

# 이때 df 값중 첫번째 index 를 가지고 온다. 

# 첫번째 index 를 지정하는 이유는 동일 위경도 값을 가진 학원들이 있기 때문이다. 
# 도로명주소 : 시군구 + 도로명 + 건물번호 입력 ex) 경기도 성남시 분당구 성남대로 349 
# 같은 건물에 있는 학원들은 동일 위경도를 가지고 있다. 

rs = rep_points.apply(lambda row: df[(df['lat']==row['lats']) & (df['lon']==row['lon'])].iloc[0], axis=1)

In [20]:
rs

,학원교습소명,학원명,개설일자,등록일자,등록상태명,정원합계,일시수용능력인원합계,분야명,교습계열명,교습과정명,도로명우편번호,도로명주소,도로명상세주소,구,lat,lon
0,학원,(2관)기찬에듀기찬수학학원,20200820,20200820,개원,240,95,입시.검정 및 보습,보통교과,보습,13555,경기도 성남시 분당구 정자일로 234,/ 305호 (정자동/ 태남프라자),분당구,37.371212,127.106282
1,학원,내가찾던영어2관학원,20100831,20100831,개원,80,58,입시.검정 및 보습,보통교과,보습,13601,경기도 성남시 분당구 불정로 258,/ 301호 (수내동/ 우진프라자),분당구,37.366635,127.127160
2,학원,(주)그래디언트학원,20111205,20111205,개원,75,72,입시.검정 및 보습,보통교과,보습,13558,경기도 성남시 분당구 성남대로331번길 3-13,/ 606호 (정자동/ 대명제스트빌딩),분당구,37.364393,127.107696
3,학원,(주)아이에스이영어원격학원,20120720,20120720,개원,27,100,입시.검정 및 보습,보통교과,보습,13591,경기도 성남시 분당구 황새울로360번길 21,/ 604호 일부 (서현동/ 분당서현신영팰리스타워),분당구,37.385710,127.125705
4,학원,(주)크레버스영재관분당제1지점학원,20140827,20140827,개원,300,269,입시.검정 및 보습,보통교과,보습,463847,경기도 성남시 분당구 성남대로 349,/ 406호/701호 (정자동/ 시그마타워),분당구,37.367295,127.107698
5,교습소,101기타음악교습소,20160419,20160419,개원,24,6,예능(대),예능(중),음악,13610,경기도 성남시 분당구 느티로 69,/ 303호 (정자동),분당구,37.368762,127.112229
6,학원,아발론랭콘수내어학원,20010322,20010322,개원,170,176,국제화,외국어,실용외국어(유아/초·중·고),13597,경기도 성남시 분당구 내정로173번길 1,/ 4층 (수내동/(주)우리은행분당지점),분당구,37.374089,127.119469
7,학원,판교엑시엄수학학원,20100702,20100702,개원,70,149,입시.검정 및 보습,보통교과,보습,463410,경기도 성남시 분당구 운중로 239,(판교동/판교신도시 근린생활시설),분당구,37.389839,127.089801
8,학원,명성플러스학원,19940722,19940722,개원,110,117,종합(대),NaN,NaN,13523,경기도 성남시 분당구 성남대로779번길 52,/ 401호/ 403호 (이매동/ 신흥코아),분당구,37.399447,127.122825
9,학원,3.14수학트레이닝센터학원,20230111,20230111,개원,1,93,입시.검정 및 보습,보통교과,보습,13597,경기도 성남시 분당구 내정로165번길 50,/ 502호 일부/ 503호 (수내동/ 크리스탈빌딩),분당구,37.377133,127.115477


In [35]:
m = folium.Map(
    location=[37.3800036, 127.1184932],
    zoom_start=15
)

coords = rs[['lat' , 'lon']]

# 반경(radius) 에 각 클러스터의 크기를 반영해줍니다 
# tooltip : 마우스 hover 시  클러스터에 포함된 개수를 표시해주도록 합니다. 

i = 0
for lat, long in zip(coords['lat'], coords['lon']):
    folium.CircleMarker([lat, long],
                    color='blue',
                    radius = len(clusters[i])/4, 
                    #popup = str(lat) + ',' + str(long),
                    tooltip = str(len(clusters[i])) ).add_to(m)
    i = i+1
m
m.save('map.html')